## 🎯 Escopo - Pipeline de Dados da Felicidade Mundial

O presente projeto tem como objetivo desenvolver um pipeline de dados completo, utilizando as camadas **Bronze**, **Silver** e **Gold**, com base no conjunto de dados do **World Happiness Report** entre os anos de **2015 a 2023**.

O foco principal é organizar, tratar, modelar e analisar as informações relativas à **felicidade mundial**, visando compreender os fatores que influenciam esse índice e possibilitar **análises exploratórias** para gerar **insights relevantes**.

---

### 🛠️ Problema a Ser Resolvido

Compreender os fatores que impactam o índice de felicidade dos países ao longo do tempo e responder perguntas-chave de negócio a partir de dados públicos.

Embora existam relatórios anuais sobre felicidade global, eles são geralmente analisados de forma estática, sem um processo sistematizado de ingestão, tratamento e exploração dinâmica. Este projeto propõe a construção de um pipeline analítico que permita:

- Avaliar a evolução da felicidade global de forma estruturada
- Identificar padrões entre países mais e menos felizes
- Relacionar indicadores socioeconômicos com níveis de felicidade

---

### 🔍 Perguntas de Negócio

A partir da modelagem dimensional e da camada Gold do pipeline, este projeto busca responder às seguintes perguntas:

1. Qual a evolução da felicidade média no mundo por ano?
2. Comparativo de felicidade média por continente em 2023?
3. Top 10 países mais felizes em 2023?
4. Fatores mais presentes entre os países mais felizes?
5. Top 10 países menos felizes em 2023
6. Quais fatores estão mais presentes entre os países menos felizes?
7. Quais são os 5 países menos felizes em cada ano?
8. Existe relação entre PIB per capita e felicidade?
9. Como a expectativa de vida afeta a felicidade?
10. Quais países se destacam por generosidade ou suporte social?
11. Correlação entre felicidade e GDP em 2023
12. Variação da felicidade no Brasil por ano?
13. Quais fatores estão mais presentes na felicidade do Brasil?
14. Comparativo: Brasil vs América Latina e Caribe

---


## 📥 Coleta do Conjunto de Dados e Persistência na Nuvem

### Fonte dos Dados

O conjunto de dados utilizado neste projeto foi obtido na plataforma Kaggle, no seguinte repositório público:

🔗 [World Happiness Report 2015–2023 – Kaggle Dataset](https://www.kaggle.com/datasets/fqayyum73/world-happiness-report-2015-2023)

A base de dados está organizada em dois arquivos principais:

1. **`WHRFinal.csv`**: Contém dados de felicidade de diversos países entre os anos de 2015 a 2023, incluindo indicadores como:
   - PIB per capita (`gdp`)
   - Suporte social (`social_support`)
   - Expectativa de vida (`life_expectancy`)
   - Liberdade (`freedom`)
   - Generosidade (`generosity`)
   - Percepção de corrupção (`corruption`)
   - Índice e ranking de felicidade

2. **`WHRContinent.csv`**: Arquivo auxiliar com a correspondência entre regiões e continentes, utilizado para enriquecer a análise geográfica.

---

### Plataforma de Armazenamento e Processamento

A plataforma utilizada foi o **Databricks Community Edition**, que fornece um ambiente gratuito baseado na nuvem para análise de dados com Spark, Python e SQL.

Etapas realizadas:

1. **Upload dos arquivos CSV** diretamente no repositório `/FileStore/tables/` do ambiente Databricks.
2. **Leitura dos dados** com `PySpark`, incluindo configuração do separador (`;`), detecção de tipos e remoção de caracteres inválidos.
3. **Persistência** dos dados em diferentes camadas do pipeline:
   - 🔶 **Bronze**: Dados brutos, conforme extraídos.
   - ⚙️ **Silver**: Dados tratados, limpos e tipados.
   - ✨ **Gold**: Modelagem dimensional para análise (tabelas fato e dimensão).
4. **Armazenamento final** no **Databricks File System (DBFS)** em formato Delta Lake, permitindo reutilização e performance em consultas SQL.

---

## Importação de Bibliotecas
- **Pandas** e **NumPy**: manipulação de dados e arrays 
- **PySpark**: para processamento distribuído (já vem embutido no Databricks)
- **Funções do PySpark**: para transformações
- **Matplotlib** e **Seaborn**:criar visualizações mais detalhadas (opcional)


In [0]:
# Bibliotecas
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, isnan, count, lit, avg


import matplotlib.pyplot as plt
import seaborn as sns


# Etapa 1 —  Camada Bronze: Ingestão de Dados

## Objetivo: 
Carregar os datasets CSV da Felicidade Mundial e mapeamento de continentes para a camada Bronze do Data Lake.

## Arquivos
- **WHRFinal.csv**: Dados de felicidade mundial (2015–2023)
- **WHRContinent.csv**: De-para de região vs. continente

## Ações
- Leitura dos arquivos CSV com PySpark
- Inferência de schema e inclusão de headers
- Salvamento no formato Delta
- Registro como tabelas no metastore Databricks


In [0]:
# Leitura do dataset principal
df_happiness = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/FileStore/tables/dataset/WHRFinal.csv")

# Leitura do dataset de continentes
df_continent = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/FileStore/tables/dataset/WHRContinent.csv")

# Visualização
display(df_happiness)
display(df_continent)


Country;Year;Region;Happiness Rank;Happiness Score;GDP;Social Support;Life Expectancy;Freedom;Generosity;Corruption
Afghanistan;2023;Southern Asia;137;1
Afghanistan;2017;Southern Asia;141;3
Afghanistan;2018;Southern Asia;145;3
Afghanistan;2022;Southern Asia;146;2
Afghanistan;2021;Southern Asia;149;2
Afghanistan;2020;Southern Asia;153;2
Afghanistan;2015;Southern Asia;153;3
Afghanistan;2016;Southern Asia;154;3
Afghanistan;2019;Southern Asia;154;3
Albania;2023;Central and Eastern Europe;83;5


Region;Continent
Southern Asia;Asia
Central and Eastern Europe;Europe
Middle East and Northern Africa;Middle East and Northern Africa
Latin America and Caribbean;Latin America and Caribbean
Australia and New Zealand;Oceania
Western Europe;Europe
Sub-Saharan Africa;Africa
Southeastern Asia;Asia
North America;North America
Eastern Asia;Asia


## Normalização dos nomes de colunas

Os arquivos CSV possuem nomes de colunas com espaços e caracteres especiais, que não são compatíveis com a criação de tabelas Delta.Será realizado:
- remover espaços e caracteres especiais
- colocar os nomes em minúsculo com underscores (`snake_case`)
- salvar od dados na camada Bronze.


In [0]:
# Função para limpar nomes de colunas
def clean_column_names(df):
    for col_name in df.columns:
        new_col = col_name.strip().lower().replace(" ", "_").replace("(", "").replace(")", "")
        df = df.withColumnRenamed(col_name, new_col)
    return df


In [0]:
# Limpar colunas dos dois DataFrames
df_happiness_clean = clean_column_names(df_happiness)
df_continent_clean = clean_column_names(df_continent)

# Visualizar novos nomes
df_happiness_clean.printSchema()
df_continent_clean.printSchema()


root
 |-- country;year;region;happiness_rank;happiness_score;gdp;social_support;life_expectancy;freedom;generosity;corruption: string (nullable = true)

root
 |-- region;continent: string (nullable = true)



In [0]:
# Salvando dataset principal
df_happiness_clean.write.format("delta").mode("overwrite").save("/mnt/bronze/happiness")
spark.sql("DROP TABLE IF EXISTS bronze_happiness")
spark.sql("""
  CREATE TABLE bronze_happiness
  USING DELTA
  LOCATION '/mnt/bronze/happiness'
""")

# Salvando dataset de continentes
df_continent_clean.write.format("delta").mode("overwrite").save("/mnt/bronze/continent")
spark.sql("DROP TABLE IF EXISTS bronze_continent")
spark.sql("""
  CREATE TABLE bronze_continent
  USING DELTA
  LOCATION '/mnt/bronze/continent'
""")


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3281497165144793>:2
      1 # Salvando dataset principal
----> 2 df_happiness_clean.write.format("delta").mode("overwrite").save("/mnt/bronze/happiness")
      3 spark.sql("DROP TABLE IF EXISTS bronze_happiness")
      4 spark.sql("""
      5   CREATE TABLE bronze_happiness
      6   USING DELTA
      7   LOCATION '/mnt/bronze/happiness'
      8 """)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1397, in DataFrameWriter.save(self, path, format, m

## Padronização de nomes de colunas

Foi identificado um erro ao tentar salvar os dados no formato Delta, devido à presença de caracteres inválidos nos nomes das colunas, será realizado:
-  função que remove qualquer caractere não alfanumérico
-  nomes convertidos para `snake_case`
-  após a limpeza, os dados serão salvos na camada Bronze


In [0]:
#remove tudo que não for letra, número ou underscore (_) dos nomes das colunas
import re
def sanitize_column_names(df):
    new_columns = []
    for col_name in df.columns:
        # Substitui espaços e caracteres especiais por underscore, remove acentos
        clean_name = re.sub(r"[^\w]", "_", col_name.strip())
        clean_name = re.sub(r"_+", "_", clean_name)  # Remove múltiplos underscores
        clean_name = clean_name.lower()
        new_columns.append(clean_name)
    
    for old_col, new_col in zip(df.columns, new_columns):
        df = df.withColumnRenamed(old_col, new_col)
    
    return df


In [0]:
# Aplicar padronização
df_happiness_clean = sanitize_column_names(df_happiness)
df_continent_clean = sanitize_column_names(df_continent)

# Verificar nomes
df_happiness_clean.printSchema()
df_continent_clean.printSchema()


root
 |-- country_year_region_happiness_rank_happiness_score_gdp_social_support_life_expectancy_freedom_generosity_corruption: string (nullable = true)

root
 |-- region_continent: string (nullable = true)



In [0]:
# Dataset principal
df_happiness_clean.write.format("delta").mode("overwrite").save("/mnt/bronze/happiness")
spark.sql("DROP TABLE IF EXISTS bronze_happiness")
spark.sql("""
  CREATE TABLE bronze_happiness
  USING DELTA
  LOCATION '/mnt/bronze/happiness'
""")

# Dataset de continentes
df_continent_clean.write.format("delta").mode("overwrite").save("/mnt/bronze/continent")
spark.sql("DROP TABLE IF EXISTS bronze_continent")
spark.sql("""
  CREATE TABLE bronze_continent
  USING DELTA
  LOCATION '/mnt/bronze/continent'
""")


Out[11]: DataFrame[]

## Validação dos dados da Camada Bronze com SQL

Após salvar os dados na camada Bronze, realizar consultas no Databricks para garantir que:

- os dados foram importados corretamente
- os tipos das colunas estão coerentes
- a quantidade de registros e países está dentro do esperado
- as tabelas podem ser usadas para análises posteriores


In [0]:
%sql
SELECT * FROM bronze_happiness LIMIT 5;


country_year_region_happiness_rank_happiness_score_gdp_social_support_life_expectancy_freedom_generosity_corruption
Afghanistan;2023;Southern Asia;137;1
Afghanistan;2017;Southern Asia;141;3
Afghanistan;2018;Southern Asia;145;3
Afghanistan;2022;Southern Asia;146;2
Afghanistan;2021;Southern Asia;149;2


In [0]:
%sql
DESCRIBE TABLE bronze_happiness;


col_name,data_type,comment
country_year_region_happiness_rank_happiness_score_gdp_social_support_life_expectancy_freedom_generosity_corruption,string,null


In [0]:
%sql
SELECT * FROM bronze_continent LIMIT 5;

region_continent
Southern Asia;Asia
Central and Eastern Europe;Europe
Middle East and Northern Africa;Middle East and Northern Africa
Latin America and Caribbean;Latin America and Caribbean
Australia and New Zealand;Oceania


In [0]:
%sql
DESCRIBE TABLE bronze_continent;


col_name,data_type,comment
region_continent,string,null


# Etapa 2 — Camada Silver: Dados Tratados

## Objetivo
Transformar os dados brutos da camada Bronze em dados limpos, padronizados e enriquecidos com informações geográficas (continente).

## Será realizado
- Remoção de registros nulos em colunas críticas
- Padronização dos nomes de país e região
- Conversão correta dos tipos de dados
- Inclusão da coluna de continente (join com bronze_continent)
- Salvamento da tabela tratada no formato Delta


In [0]:
from pyspark.sql.functions import col, trim

# Carregar as tabelas da Bronze
df_bronze_happiness = spark.read.format("delta").load("/mnt/bronze/happiness")
df_bronze_continent = spark.read.format("delta").load("/mnt/bronze/continent")

# Limpeza e tratamento (Silver)
df_silver = (
    df_bronze_happiness
    .dropna(subset=["country", "year", "happiness_score"])
    .withColumn("country", trim(col("country")))
    .withColumn("region", trim(col("region")))
    .withColumn("year", col("year").cast("int"))
    .withColumn("happiness_score", col("happiness_score").cast("float"))
    .withColumn("gdp", col("gdp").cast("float"))
    .withColumn("social_support", col("social_support").cast("float"))
    .withColumn("life_expectancy", col("life_expectancy").cast("float"))
    .withColumn("freedom", col("freedom").cast("float"))
    .withColumn("generosity", col("generosity").cast("float"))
    .withColumn("corruption", col("corruption").cast("float"))
)

# Enriquecimento com continente
df_silver = df_silver.join(
    df_bronze_continent,
    on="region",
    how="left"
)


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3281497165144823>:9
      5 df_bronze_continent = spark.read.format("delta").load("/mnt/bronze/continent")
      7 # Limpeza e tratamento (Silver)
      8 df_silver = (
----> 9     df_bronze_happiness
     10     .dropna(subset=["country", "year", "happiness_score"])
     11     .withColumn("country", trim(col("country")))
     12     .withColumn("region", trim(col("region")))
     13     .withColumn("year", col("year").cast("int"))
     14     .withColumn("happiness_score", col("happiness_score").cast("float"))
     15     .withColumn("gdp", col("gdp").cast("float"))
     16     .withColumn("social_support", col("social_support").cast("float"))
     17     .withColumn("life_expectancy", col("life_expectancy").cast("float"))
     18     .withColumn("freedom", col("freedom").cast("float"))
     19     .withColumn("generosit

In [0]:
# Corrigir o erro (como se todo o cabeçalho do CSV tivesse sido lido como um único campo)
# Refazer a leitura da Bronze
df_bronze_happiness = spark.read.format("csv") \
    .option("header", "true") \
    .option("sep", ",") \
    .option("inferSchema", "true") \
    .load("/FileStore/tables/dataset/WHRFinal.csv")

df_bronze_continent = spark.read.format("csv") \
    .option("header", "true") \
    .option("sep", ",") \
    .option("inferSchema", "true") \
    .load("/FileStore/tables/dataset/WHRContinent.csv")


In [0]:
#Verifique se as colunas estão no formato correto
df_bronze_happiness.printSchema()
df_bronze_happiness.show(5)


root
 |-- Country;Year;Region;Happiness Rank;Happiness Score;GDP;Social Support;Life Expectancy;Freedom;Generosity;Corruption: string (nullable = true)

+-------------------------------------------------------------------------------------------------------------------+
|Country;Year;Region;Happiness Rank;Happiness Score;GDP;Social Support;Life Expectancy;Freedom;Generosity;Corruption|
+-------------------------------------------------------------------------------------------------------------------+
|                                                                                               Afghanistan;2023;...|
|                                                                                               Afghanistan;2017;...|
|                                                                                               Afghanistan;2018;...|
|                                                                                               Afghanistan;2022;...|
|                    

In [0]:
# Reaplicar a função de padronização nas colunas
df_bronze_happiness = sanitize_column_names(df_bronze_happiness)
df_bronze_continent = sanitize_column_names(df_bronze_continent)


In [0]:
# Especificar o separador correto (;) na leitura do CSV
# Lendo corretamente com separador ";"
df_bronze_happiness = spark.read.format("csv") \
    .option("header", "true") \
    .option("sep", ";") \
    .option("inferSchema", "true") \
    .load("/FileStore/tables/dataset/WHRFinal.csv")

df_bronze_continent = spark.read.format("csv") \
    .option("header", "true") \
    .option("sep", ";") \
    .option("inferSchema", "true") \
    .load("/FileStore/tables/dataset/WHRContinent.csv")


In [0]:
df_bronze_happiness.printSchema()
df_bronze_happiness.show(5)


root
 |-- Country: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Happiness Rank: integer (nullable = true)
 |-- Happiness Score: string (nullable = true)
 |-- GDP: string (nullable = true)
 |-- Social Support: string (nullable = true)
 |-- Life Expectancy: string (nullable = true)
 |-- Freedom: string (nullable = true)
 |-- Generosity: string (nullable = true)
 |-- Corruption: string (nullable = true)

+-----------+----+-------------+--------------+---------------+-----+--------------+---------------+-------+----------+----------+
|    Country|Year|       Region|Happiness Rank|Happiness Score|  GDP|Social Support|Life Expectancy|Freedom|Generosity|Corruption|
+-----------+----+-------------+--------------+---------------+-----+--------------+---------------+-------+----------+----------+
|Afghanistan|2023|Southern Asia|           137|          1,859|0,645|         0,000|          0,087|  0,000|     0,093|     0,059|
|Afghanista

In [0]:
#sParonizar as colunas
df_bronze_happiness = sanitize_column_names(df_bronze_happiness)
df_bronze_continent = sanitize_column_names(df_bronze_continent)


In [0]:
#Trocar vírgula por ponto nos números
from pyspark.sql.functions import regexp_replace, col, trim

# Lista das colunas que estão com números no formato string (com vírgula)
cols_to_clean = [
    "happiness_score", "gdp", "social_support",
    "life_expectancy", "freedom", "generosity", "corruption"
]

# Substituir vírgula por ponto e converter para float
for c in cols_to_clean:
    df_bronze_happiness = df_bronze_happiness.withColumn(
        c, regexp_replace(col(c), ",", ".").cast("float")
    )

# Padronizar colunas de texto
df_bronze_happiness = df_bronze_happiness.withColumn("country", trim(col("country")))
df_bronze_happiness = df_bronze_happiness.withColumn("region", trim(col("region")))


In [0]:
# Join com a tabela de continentes
df_silver = df_bronze_happiness.join(
    df_bronze_continent,
    on="region",
    how="left"
)



In [0]:
df_silver.select("country", "region", "continent", "year", "happiness_score").show(5)


+-----------+-------------+---------+----+---------------+
|    country|       region|continent|year|happiness_score|
+-----------+-------------+---------+----+---------------+
|Afghanistan|Southern Asia|     Asia|2023|          1.859|
|Afghanistan|Southern Asia|     Asia|2017|          3.794|
|Afghanistan|Southern Asia|     Asia|2018|          3.632|
|Afghanistan|Southern Asia|     Asia|2022|          2.404|
|Afghanistan|Southern Asia|     Asia|2021|          2.523|
+-----------+-------------+---------+----+---------------+
only showing top 5 rows



In [0]:
# Salvar como Delta
df_silver.write.format("delta").mode("overwrite").save("/mnt/silver/happiness")

# Registrar a tabela no catálogo
spark.sql("DROP TABLE IF EXISTS silver_happiness")
spark.sql("""
  CREATE TABLE silver_happiness
  USING DELTA
  LOCATION '/mnt/silver/happiness'
""")


Out[35]: DataFrame[]

## Finalização da Camada Silver

Dados limpos e enriquecidos com a informação de continente, realizado:

- join entre bronze_happiness e bronze_continent usando a coluna `region`
- verificação de consistência dos dados após o join
- salvamento da tabela `silver_happiness` no formato Delta


# Etapa 3 — Camada Gold: Modelagem Estrela

## Objetivo
Criar um modelo dimensional (estrela) para suportar análises exploratórias dos dados de felicidade mundial, considerando aspectos geográficos, temporais e indicadores sociais.

## Estrutura
- **Fato**: fato_felicidade — Contém os indicadores por país e ano
- **Dimensões**:
  - dim_pais — Informações geográficas: país, região, continente
  - dim_tempo — Informações temporais: ano, possíveis agregações futuras



In [0]:
# Dimensão País — dim_pais
#Contém informações geográficas para análise por país, região e continente. Cada país recebe um ID único (id_pais) para ser usado como chave na tabela fato.
# Colunas:id_pais (PK), country, region, continent

from pyspark.sql.functions import monotonically_increasing_id

# Selecionar apenas colunas geográficas distintas
dim_pais = df_silver.select("country", "region", "continent").dropDuplicates()

# Criar coluna de ID único para cada país
dim_pais = dim_pais.withColumn("id_pais", monotonically_increasing_id())

# Visualizar
dim_pais.orderBy("country").show(10, truncate=False)


+-----------+-------------------------------+-------------------------------+-------+
|country    |region                         |continent                      |id_pais|
+-----------+-------------------------------+-------------------------------+-------+
|Afghanistan|Southern Asia                  |Asia                           |30     |
|Albania    |Central and Eastern Europe     |Europe                         |25     |
|Algeria    |Middle East and Northern Africa|Middle East and Northern Africa|109    |
|Angola     |Sub-Saharan Africa             |Africa                         |98     |
|Argentina  |Latin America and Caribbean    |Latin America and Caribbean    |106    |
|Armenia    |Central and Eastern Europe     |Europe                         |138    |
|Australia  |Australia and New Zealand      |Oceania                        |57     |
|Austria    |Western Europe                 |Europe                         |68     |
|Azerbaijan |Central and Eastern Europe     |Europe   

In [0]:
dim_pais.write.format("delta").mode("overwrite").save("/mnt/gold/dim_pais")

spark.sql("DROP TABLE IF EXISTS dim_pais")
spark.sql("""
  CREATE TABLE dim_pais
  USING DELTA
  LOCATION '/mnt/gold/dim_pais'
""")


Out[37]: DataFrame[]

In [0]:
# Dimensão Tempo — dim_tempo
#Contém os anos em que foram registrados os dados, com um identificador único (id_tempo).
# Colunas: id_tempo(PK), year


# Selecionar anos únicos
dim_tempo = df_silver.select("year").dropDuplicates().orderBy("year")

# Adicionar id_tempo
dim_tempo = dim_tempo.withColumn("id_tempo", monotonically_increasing_id())

# Visualizar
dim_tempo.show()


+----+--------+
|year|id_tempo|
+----+--------+
|2015|       0|
|2016|       1|
|2017|       2|
|2018|       3|
|2019|       4|
|2020|       5|
|2021|       6|
|2022|       7|
|2023|       8|
+----+--------+



In [0]:
dim_tempo.write.format("delta").mode("overwrite").save("/mnt/gold/dim_tempo")

spark.sql("DROP TABLE IF EXISTS dim_tempo")
spark.sql("""
  CREATE TABLE dim_tempo
  USING DELTA
  LOCATION '/mnt/gold/dim_tempo'
""")


Out[39]: DataFrame[]

In [0]:
# Join com dim_pais
df_fato = df_silver.join(
    dim_pais,
    on=["country", "region", "continent"],
    how="left"
)

# Join com dim_tempo
df_fato = df_fato.join(
    dim_tempo,
    on="year",
    how="left"
)

# Selecionar colunas da fato
fato_felicidade = df_fato.select(
    "id_pais", "id_tempo",
    "happiness_score",
    "happiness_rank",
    "gdp",
    "social_support",
    "life_expectancy",
    "freedom",
    "generosity",
    "corruption"
)

# Visualizar
fato_felicidade.show(5)


+-------+--------+---------------+--------------+-----+--------------+---------------+-------+----------+----------+
|id_pais|id_tempo|happiness_score|happiness_rank|  gdp|social_support|life_expectancy|freedom|generosity|corruption|
+-------+--------+---------------+--------------+-----+--------------+---------------+-------+----------+----------+
|     30|       8|          1.859|           137|0.645|           0.0|          0.087|    0.0|     0.093|     0.059|
|     30|       2|          3.794|           141|0.401|         0.582|          0.181|  0.106|     0.312|     0.061|
|     30|       3|          3.632|           145|0.332|         0.537|          0.255|  0.085|     0.191|     0.036|
|     30|       7|          2.404|           146|0.758|           0.0|          0.289|    0.0|     0.089|     0.005|
|     30|       6|          2.523|           149| 0.37|           0.0|          0.126|    0.0|     0.122|      0.01|
+-------+--------+---------------+--------------+-----+---------

## Tabela Fato — fato_felicidade

Tabela central do modelo estrela. Contém os indicadores de felicidade associados a um país e um ano, usando chaves para as dimensões.

### Colunas:
- id_pais (FK)
- id_tempo (FK)
- happiness_score
- happiness_rank
- gdp
- social_support
- life_expectancy
- freedom
- generosity
- corruption


In [0]:
fato_felicidade.write.format("delta").mode("overwrite").save("/mnt/gold/fato_felicidade")

spark.sql("DROP TABLE IF EXISTS fato_felicidade")
spark.sql("""
  CREATE TABLE fato_felicidade
  USING DELTA
  LOCATION '/mnt/gold/fato_felicidade'
""")


Out[53]: DataFrame[]

In [0]:
# Criar views temporárias
spark.table("fato_felicidade").createOrReplaceTempView("vw_fato_felicidade")
spark.table("dim_pais").createOrReplaceTempView("vw_dim_pais")
spark.table("dim_tempo").createOrReplaceTempView("vw_dim_tempo")


## 📊 Resumo das Perguntas e Respostas (Análises)

São 14 perguntas exploradas com base no dataset World Happiness Report (2015–2023), utilizando SQL e modelagem dimensional.

| Nº | Pergunta                                                                                 | Métrica Principal                                       |
|----|------------------------------------------------------------------------------------------|---------------------------------------------------------|
| 1  | Qual a evolução da felicidade média no mundo por ano?                                   | AVG(`happiness_score`) por ano                          |
| 2  | Comparativo de felicidade média por continente em 2023?                                 | AVG(`happiness_score`) por continente                   |
| 3  | Top 10 países mais felizes em 2023?                                                      | TOP 10 `happiness_score`                                |
| 4  | Fatores mais presentes entre os países mais felizes?                                    | Médias dos indicadores (`happiness_score` ≥ 7)          |
| 5  | Top 10 países menos felizes em 2023                                                      | Menores `happiness_score`                               |
| 6  | Quais fatores estão mais presentes entre os países menos felizes?                       | Médias dos indicadores (`happiness_score` ≤ 4)          |
| 7  | Quais são os 5 países menos felizes em cada ano?                                         | `ROW_NUMBER()` por ano (`happiness_score` ASC)          |
| 8  | Existe relação entre PIB per capita e felicidade?                                       | Dispersão entre `gdp` e `happiness_score`               |
| 9  | Como a expectativa de vida afeta a felicidade?                                           | Dispersão entre `life_expectancy` e `happiness_score`   |
| 10 | Quais países se destacam por generosidade ou suporte social?                            | AVG(`generosity`), AVG(`social_support`) por país       |
| 11 | Correlação entre felicidade e GDP em 2023                                                | `happiness_score` vs `gdp` (somente 2023)               |
| 12 | Variação da felicidade no Brasil por ano?                                                | Histórico de `happiness_score` do Brasil                |
| 13 | Quais fatores estão mais presentes na felicidade do Brazil?                             | Médias dos indicadores do Brasil                        |
| 14 | Comparativo: Brasil vs América Latina e Caribe                                          | Médias comparativas Brasil x América Latina             |


In [0]:
%sql
SELECT t.year, ROUND(AVG(f.happiness_score), 2) AS media_happiness
FROM vw_fato_felicidade f
JOIN vw_dim_tempo t ON f.id_tempo = t.id_tempo
GROUP BY t.year
ORDER BY t.year;


year,media_happiness
2015,5.38
2016,5.38
2017,5.35
2018,5.37
2019,5.41
2020,5.47
2021,5.53
2022,5.55
2023,5.54


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT p.continent, ROUND(AVG(f.happiness_score), 2) AS media_happiness
FROM vw_fato_felicidade f
JOIN vw_dim_pais p ON f.id_pais = p.id_pais
JOIN vw_dim_tempo t ON f.id_tempo = t.id_tempo
WHERE t.year = 2023
GROUP BY p.continent
ORDER BY media_happiness DESC;


continent,media_happiness
Oceania,7.11
North America,6.93
Europe,6.36
Latin America and Caribbean,5.97
Asia,5.19
Middle East and Northern Africa,5.1
Africa,4.38


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT p.country, f.happiness_score
FROM vw_fato_felicidade f
JOIN vw_dim_pais p ON f.id_pais = p.id_pais
JOIN vw_dim_tempo t ON f.id_tempo = t.id_tempo
WHERE t.year = 2023
ORDER BY f.happiness_score DESC
LIMIT 10;


country,happiness_score
Finland,7.804
Denmark,7.586
Iceland,7.53
Israel,7.473
Netherlands,7.403
Sweden,7.395
Norway,7.315
Switzerland,7.24
Luxembourg,7.228
New Zealand,7.123


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT 'GDP' AS fator, ROUND(AVG(gdp), 3) AS media_valor FROM vw_fato_felicidade WHERE happiness_score >= 7
UNION ALL
SELECT 'Social Support', ROUND(AVG(social_support), 3) FROM vw_fato_felicidade WHERE happiness_score >= 7
UNION ALL
SELECT 'Life Expectancy', ROUND(AVG(life_expectancy), 3) FROM vw_fato_felicidade WHERE happiness_score >= 7
UNION ALL
SELECT 'Freedom', ROUND(AVG(freedom), 3) FROM vw_fato_felicidade WHERE happiness_score >= 7
UNION ALL
SELECT 'Generosity', ROUND(AVG(generosity), 3) FROM vw_fato_felicidade WHERE happiness_score >= 7
UNION ALL
SELECT 'Corruption', ROUND(AVG(corruption), 3) FROM vw_fato_felicidade WHERE happiness_score >= 7;


fator,media_valor
GDP,1.494
Social Support,1.343
Life Expectancy,0.845
Freedom,0.616
Generosity,0.283
Corruption,0.322


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT p.country, f.happiness_score
FROM vw_fato_felicidade f
JOIN vw_dim_pais p ON f.id_pais = p.id_pais
JOIN vw_dim_tempo t ON f.id_tempo = t.id_tempo
WHERE t.year = 2023
ORDER BY f.happiness_score ASC
LIMIT 10;


country,happiness_score
Afghanistan,1.859
Lebanon,2.392
Sierra Leone,3.138
Zimbabwe,3.204
Congo (Kinshasa),3.207
Botswana,3.435
Malawi,3.495
Comoros,3.545
Tanzania,3.694
Zambia,3.982


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT 'GDP' AS fator, ROUND(AVG(gdp), 3) AS media_valor FROM vw_fato_felicidade WHERE happiness_score <= 4
UNION ALL
SELECT 'Social Support', ROUND(AVG(social_support), 3) FROM vw_fato_felicidade WHERE happiness_score <= 4
UNION ALL
SELECT 'Life Expectancy', ROUND(AVG(life_expectancy), 3) FROM vw_fato_felicidade WHERE happiness_score <= 4
UNION ALL
SELECT 'Freedom', ROUND(AVG(freedom), 3) FROM vw_fato_felicidade WHERE happiness_score <= 4
UNION ALL
SELECT 'Generosity', ROUND(AVG(generosity), 3) FROM vw_fato_felicidade WHERE happiness_score <= 4
UNION ALL
SELECT 'Corruption', ROUND(AVG(corruption), 3) FROM vw_fato_felicidade WHERE happiness_score <= 4;


fator,media_valor
GDP,0.46
Social Support,0.638
Life Expectancy,0.29
Freedom,0.325
Generosity,0.208
Corruption,0.118


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
WITH paises_menos_felizes AS (
  SELECT 
    t.year,
    p.country,
    f.happiness_score,
    ROW_NUMBER() OVER (PARTITION BY t.year ORDER BY f.happiness_score ASC) AS posicao
  FROM vw_fato_felicidade f
  JOIN vw_dim_pais p ON f.id_pais = p.id_pais
  JOIN vw_dim_tempo t ON f.id_tempo = t.id_tempo
  WHERE f.happiness_score IS NOT NULL
)

SELECT year, country, happiness_score
FROM paises_menos_felizes
WHERE posicao <= 5
ORDER BY year, happiness_score;



year,country,happiness_score
2015,Togo,2.839
2015,Burundi,2.905
2015,Syria,3.006
2015,Benin,3.34
2015,Rwanda,3.465
2016,Burundi,2.905
2016,Syria,3.069
2016,Togo,3.303
2016,Afghanistan,3.36
2016,Benin,3.484


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
--Quanto maior o PIB per capita, maior tende a ser a felicidade?
SELECT f.happiness_score, f.gdp
FROM vw_fato_felicidade f
JOIN vw_dim_tempo t ON f.id_tempo = t.id_tempo
WHERE f.gdp IS NOT NULL AND f.happiness_score IS NOT NULL


happiness_score,gdp
1.859,0.645
3.794,0.401
3.632,0.332
2.404,0.758
2.523,0.37
2.567,0.301
3.575,0.32
3.36,0.382
3.203,0.35
5.277,1.449


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT f.happiness_score, f.life_expectancy
FROM vw_fato_felicidade f
JOIN vw_dim_tempo t ON f.id_tempo = t.id_tempo
WHERE f.life_expectancy IS NOT NULL AND f.happiness_score IS NOT NULL


happiness_score,life_expectancy
1.859,0.087
3.794,0.181
3.632,0.255
2.404,0.289
2.523,0.126
2.567,0.266
3.575,0.303
3.36,0.173
3.203,0.361
5.277,0.48


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT p.country, ROUND(AVG(f.generosity), 3) AS media_generosity,
       ROUND(AVG(f.social_support), 3) AS media_suporte
FROM vw_fato_felicidade f
JOIN vw_dim_pais p ON f.id_pais = p.id_pais
GROUP BY p.country
ORDER BY media_generosity DESC
LIMIT 10;


country,media_generosity,media_suporte
Myanmar,0.618,0.961
Indonesia,0.516,1.072
Somaliland Region,0.515,0.759
Haiti,0.453,0.557
Bhutan,0.435,1.202
Syria,0.431,0.356
Thailand,0.425,1.256
United Kingdom,0.378,1.319
Malta,0.377,1.341
Gambia,0.376,0.714


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT f.happiness_score, f.gdp
FROM vw_fato_felicidade f
JOIN vw_dim_tempo t ON f.id_tempo = t.id_tempo
WHERE t.year = 2023


happiness_score,gdp
1.859,0.645
5.277,1.449
5.329,1.353
6.024,1.59
5.342,1.466
7.095,1.899
7.097,1.927
6.173,1.883
4.282,1.133
6.859,1.907


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT t.year, f.happiness_score
FROM vw_fato_felicidade f
JOIN vw_dim_pais p ON f.id_pais = p.id_pais
JOIN vw_dim_tempo t ON f.id_tempo = t.id_tempo
WHERE p.country = 'Brazil'
ORDER BY t.year;


year,happiness_score
2015,6.983
2016,6.952
2017,6.635
2018,6.419
2019,6.3
2020,6.376
2021,6.33
2022,6.293
2023,6.125


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT
  'GDP' AS fator, ROUND(AVG(gdp), 3) AS media_valor
FROM vw_fato_felicidade f
JOIN vw_dim_pais p ON f.id_pais = p.id_pais
WHERE p.country = 'Brazil'

UNION ALL
SELECT 'Social Support', ROUND(AVG(social_support), 3)
FROM vw_fato_felicidade f JOIN vw_dim_pais p ON f.id_pais = p.id_pais
WHERE p.country = 'Brazil'

UNION ALL
SELECT 'Life Expectancy', ROUND(AVG(life_expectancy), 3)
FROM vw_fato_felicidade f JOIN vw_dim_pais p ON f.id_pais = p.id_pais
WHERE p.country = 'Brazil'

UNION ALL
SELECT 'Freedom', ROUND(AVG(freedom), 3)
FROM vw_fato_felicidade f JOIN vw_dim_pais p ON f.id_pais = p.id_pais
WHERE p.country = 'Brazil'

UNION ALL
SELECT 'Generosity', ROUND(AVG(generosity), 3)
FROM vw_fato_felicidade f JOIN vw_dim_pais p ON f.id_pais = p.id_pais
WHERE p.country = 'Brazil'

UNION ALL
SELECT 'Corruption', ROUND(AVG(corruption), 3)
FROM vw_fato_felicidade f JOIN vw_dim_pais p ON f.id_pais = p.id_pais
WHERE p.country = 'Brazil';


fator,media_valor
GDP,1.118
Social Support,1.246
Life Expectancy,0.638
Freedom,0.479
Generosity,0.135
Corruption,0.122


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- MÉDIAS DO BRASIL
SELECT 'Brazil' AS grupo, 'GDP' AS fator, ROUND(AVG(gdp), 3) AS media_valor
FROM vw_fato_felicidade f
JOIN vw_dim_pais p ON f.id_pais = p.id_pais
WHERE p.country = 'Brazil'
UNION ALL
SELECT 'Brazil', 'Social Support', ROUND(AVG(social_support), 3)
FROM vw_fato_felicidade f JOIN vw_dim_pais p ON f.id_pais = p.id_pais WHERE p.country = 'Brazil'
UNION ALL
SELECT 'Brazil', 'Life Expectancy', ROUND(AVG(life_expectancy), 3)
FROM vw_fato_felicidade f JOIN vw_dim_pais p ON f.id_pais = p.id_pais WHERE p.country = 'Brazil'
UNION ALL
SELECT 'Brazil', 'Freedom', ROUND(AVG(freedom), 3)
FROM vw_fato_felicidade f JOIN vw_dim_pais p ON f.id_pais = p.id_pais WHERE p.country = 'Brazil'
UNION ALL
SELECT 'Brazil', 'Generosity', ROUND(AVG(generosity), 3)
FROM vw_fato_felicidade f JOIN vw_dim_pais p ON f.id_pais = p.id_pais WHERE p.country = 'Brazil'
UNION ALL
SELECT 'Brazil', 'Corruption', ROUND(AVG(corruption), 3)
FROM vw_fato_felicidade f JOIN vw_dim_pais p ON f.id_pais = p.id_pais WHERE p.country = 'Brazil'

-- MÉDIAS DA AMÉRICA LATINA
UNION ALL
SELECT 'América Latina', 'GDP', ROUND(AVG(gdp), 3)
FROM vw_fato_felicidade f JOIN vw_dim_pais p ON f.id_pais = p.id_pais
WHERE p.region = 'Latin America and Caribbean'
UNION ALL
SELECT 'América Latina', 'Social Support', ROUND(AVG(social_support), 3)
FROM vw_fato_felicidade f JOIN vw_dim_pais p ON f.id_pais = p.id_pais
WHERE p.region = 'Latin America and Caribbean'
UNION ALL
SELECT 'América Latina', 'Life Expectancy', ROUND(AVG(life_expectancy), 3)
FROM vw_fato_felicidade f JOIN vw_dim_pais p ON f.id_pais = p.id_pais
WHERE p.region = 'Latin America and Caribbean'
UNION ALL
SELECT 'América Latina', 'Freedom', ROUND(AVG(freedom), 3)
FROM vw_fato_felicidade f JOIN vw_dim_pais p ON f.id_pais = p.id_pais
WHERE p.region = 'Latin America and Caribbean'
UNION ALL
SELECT 'América Latina', 'Generosity', ROUND(AVG(generosity), 3)
FROM vw_fato_felicidade f JOIN vw_dim_pais p ON f.id_pais = p.id_pais
WHERE p.region = 'Latin America and Caribbean'
UNION ALL
SELECT 'América Latina', 'Corruption', ROUND(AVG(corruption), 3)
FROM vw_fato_felicidade f JOIN vw_dim_pais p ON f.id_pais = p.id_pais
WHERE p.region = 'Latin America and Caribbean';


grupo,fator,media_valor
Brazil,GDP,1.118
Brazil,Social Support,1.246
Brazil,Life Expectancy,0.638
Brazil,Freedom,0.479
Brazil,Generosity,0.135
Brazil,Corruption,0.122
América Latina,GDP,1.017
América Latina,Social Support,1.147
América Latina,Life Expectancy,0.648
América Latina,Freedom,0.505


Databricks visualization. Run in Databricks to view.

## Autoavaliação do Projeto

Ao longo do desenvolvimento deste projeto, foi possível aplicar de forma prática os conceitos de pipeline de dados, organização por camadas (Bronze, Silver e Gold), modelagem dimensional e análises exploratórias com foco em resolução de problemas de negócio.

---

### ✅ Objetivos Alcançados

- Construção do pipeline completo no ambiente Databricks Community Edition, com todas as camadas operacionais (Bronze → Silver → Gold).
- Leitura, limpeza e padronização de dados reais, provenientes de um dataset público da plataforma Kaggle.
- Modelagem estrela, com separação clara entre fatos e dimensões, permitindo consultas analíticas robustas.
- Criação de 14 perguntas de negócio relevantes que exploram aspectos sociais, econômicos e geográficos da felicidade mundial.
- Geração de visualizações no Databricks para facilitar a interpretação dos dados.

----

### ⚠️ Desafios

- Dificuldades iniciais no uso do Databricks, especialmente para entender o funcionamento dos clusters, execução de notebooks e gestão dos arquivos no DBFS.
- Desconexão frequente do cluster por inatividade, exigindo reativação e carga dos blocos de código.
- Necessidade de lidar com problemas na leitura dos arquivos CSV, como separadores incorretos e conversão de tipos.
- Aprendizado gradual sobre como aplicar a modelagem dimensional na prática, incluindo criação de tabelas fato e dimensão com joins.
- A construção do processo de ETL exigiu buscar novos conhecimentos, especialmente sobre tratamento de dados com PySpark e a lógica por trás das camadas Bronze, Silver e Gold — o que trouxe muito aprendizado técnico e prático.

---

### 🎯 Atingimento dos Objetivos

Todos os objetivos traçados no início do trabalho foram atingidos.

- O pipeline foi construído com todas as suas camadas funcionais.
- Os dados foram tratados, enriquecidos e organizados em uma modelagem estrela.
- Todas as perguntas de negócio foram respondidas com base nos dados disponíveis.
- As visualizações permitiram compreender padrões e gerar insights relevantes sobre a felicidade mundial.

---

### 🧭 Reflexão Final

O projeto foi extremamente valioso e trouxe muito aprendizado — tanto no aspecto técnico quanto na organização de um raciocínio analítico orientado por dados.

Mesmo com os desafios iniciais, especialmente ligados ao uso da plataforma em nuvem, foi possível implementar todas as etapas do pipeline com sucesso. A construção do ETL, em especial, exigiu pesquisa e prática, resultando em aprendizado.

Através da modelagem e das análises, consegui obter respostas claras e fundamentadas para todas as perguntas de negócio propostas, extraindo insights significativos sobre os fatores que influenciam a felicidade ao redor do mundo.

Além disso, este projeto representou um reencontro com a programação, uma área pela qual tenho grande afinidade e que me trouxe satisfação ao longo do desenvolvimento. Vislumbro, a partir desta experiência, a possibilidade de uma mudança de carreira, com foco maior em dados, tecnologia e soluções analíticas — algo que me motiva e inspira para os próximos passos profissionais.
